In [1]:
import sys
import cython
import numpy as np
import scipy.sparse as sps

sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()

In [3]:
import os
output_folder_path = "../result_experiments/SLIM+KNN_sim/"

In [4]:
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
        
n_cases = 50  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [5]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

In [6]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
#evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13644 ( 0.0%) Users that have less than 1 test interactions


In [7]:
# We obtain the Similarity Matrix of the best models
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender2 = ItemKNNCBFRecommender(URM_train, ICM_genre_all)

ItemKNNCBFRecommender: ICM Detected 559 ( 3.1%) items with no features.


In [8]:
# We fit the models with best hyperparameters found
hyp_best_1 = {'topK': 1203, 'l1_ratio': 0.0007860291403064104, 'alpha': 0.07969416051151235, 'workers': 2}
hyp_best_2 = {"shrink": 1642, "topK": 734, "feature_weighting": "TF-IDF", "normalize": True}

recommender1.fit(**hyp_best_1)
recommender2.fit(**hyp_best_2)

100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [42:57<00:00,  7.00it/s]


Similarity column 18059 (100.0%), 5182.94 column/sec. Elapsed time 3.48 sec


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# We then draw the heatmaps
slim_sim = recommender1.W_sparse
knn_sim = recommender2.W_sparse

slim_sim_max = slim_sim.max()
knn_sim_max = knn_sim.max()
vmax = np.max([slim_sim_max, knn_sim_max])

f, [ax1, ax2] = plt.subplots(1, 2, figsize=(16, 9), sharey=True)
sns.heatmap(slim_sim.todense(), cbar=False, vmin=0, vmax=vmax, ax=ax1, cmap="Reds")
sns.heatmap(knn_sim.todense(), cbar=False, vmin=0, vmax=vmax, ax=ax2, cmap="Reds")
ax1.xaxis.set_ticks_position("top")
ax2.xaxis.set_ticks_position("top")